In [ ]:
from __future__ import print_function
import os, warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys 
import gzip
import glob
import json
import time 
import argparse
import numpy as np
import pandas as pd
from pandas import json_normalize
from IPython.core.display import Image
import tensorflow as tf
from tensorflow import keras
from pandas import json_normalize
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from tensorflow.keras.utils import to_categorical
from keras import backend as K
from keras.layers import Input, Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten
from keras import optimizers
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from keras.models import Sequential, Model, model_from_json, model_from_yaml
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau

In [ ]:
genotype_input = pd.read_csv("<path_to_data>/data_213_WIT_AA_CAU_AS.csv", low_memory=False, sep=",")
genotype_input.iloc[0:3,[0,1,2]]

In [ ]:
genotype_input.shape

In [ ]:
###getting the sample IDs
cols = genotype_input.columns[1:].values.tolist()
print("Length of cols variable: ",len(cols))
print("First few Submitters_ID", cols[0:4])

In [ ]:
cols 
type(cols)
with open('<path_to_data>/G_sample_names_all.tsv', 'w') as f:
    for item in cols:
        #print (item)
        f.write("%s\n" % item)

In [ ]:
# Transpose the data
genotype_input_t = genotype_input.T
genotype_input_t.shape

In [ ]:
genotype_input_t.iloc[0:7, 0:3]

In [ ]:
# remove the top row and save the output
genotype_input_t = genotype_input_t.drop(genotype_input_t.index[0:1], axis=0)
genotype_input_t.shape

In [ ]:
genotype_input_t.iloc[0:3, 0:2]

In [ ]:
genotype_input_t = genotype_input_t.reset_index(drop=True)

In [ ]:
# Drop the index (sample name row)
features = genotype_input_t.drop(genotype_input_t.index[:0], axis=1)
features.iloc[0:3, 0:2]

In [ ]:
features.to_csv('<path_to_data>/G_Test_all-datastypes.tsv', sep='\t', index=False)
df = pd.read_csv("<path_to_data>/G_Test_all-datastypes.tsv", sep = "\t", low_memory = False)
df.head()

In [ ]:
print("There are {} samples.".format(df.shape[0]))
print("There are {} variants.".format(df.shape[1]))

In [ ]:
##Initialize Model with 3 classes, CAU, AA, AS
def create_model(num_classes):
    """
    Creates model using number of genes as input and the number of cancer types as output.
    
    Parameters: 
    
    Returns: model
    """
    filters = 128
    filter_len = 20
    stride = 1

    model = Sequential()
    model.add(Conv1D(filters = filters,
                     kernel_size = filter_len,
                     strides = stride,
                     padding='valid',
                     input_shape=(213, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 1))
    model.add(Conv1D(filters = filters,
                     kernel_size = filter_len,
                     strides = stride,
                     padding = 'valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size = 10))
    model.add(Flatten())
    model.add(Dense(200))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(20))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model

In [ ]:
#Choose numClasses, in this case we have 3 classes: CAU, AA, AS.
#For 2 classes change numClasses=2
numClasses=3
selectedModel = create_model(numClasses)
selectedModel.summary()

In [ ]:
#Load model
weight_mod='/<path_to_data>/RCNN_213_AA_CAU_AS.model.h5'

In [ ]:
def load_trained_weights(weight_mod, model):
    """
    Load trained weights to the model based on the number of genes and background.
    
    """
    weights_path = '/<path_to_data>/RCNN_213_AA_CAU_AS.model.h5'
    
    model.load_weights(weights_path)
    
    print("Loading weights from: {}".format(weights_path))
    
    return model

In [ ]:
#Perfom admixture analysis using selectd model
selectedModel = load_trained_weights(weight_mod, selectedModel)
data = np.expand_dims(df, axis=2)
predictions = selectedModel.predict(data)

In [ ]:
#Admixture probablity Prediction ising admix-AI
labels_list =  ['AA',
                    'AS',
                   'CAU']
pred_df = pd.DataFrame(predictions,columns = labels_list)
pred_df["predicted_class"] = pred_df.iloc[:,0:numClasses].idxmax(axis = 1)
pred_df["probability"] = pred_df.iloc[:,0:numClasses].max(axis = 1)
pred_df.head(10)

In [ ]:
#Input Sample names with Header (Input #2)
sample_names = pd.read_table('/<path_to_data>/G_sample_names_all copy.tsv')
sample_names

In [ ]:
#Concatenate admixture probabilities to Sample names
Final_pred=pred_df
Final_pred['Sample_names']=sample_names.copy()
Final_Pred

In [ ]:
#Write final Results to csv
Final_pred.to_csv('/<path_to_data>/predictions_WITH_AA_CAU_AS.csv')